# Build Simple word2vec Model -- Taney


* Code Sampled from [Laura K. Nelson's GitHub](https://github.com/lknelson/measuring_intersectionality/blob/main/scripts/00_measuringintersectionality_constructmodels.ipynb)

In [1]:
import sys, os, string, glob, gensim, warnings
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize

# Read in config.py (git ignored file) for API username and pw.
config_path = os.path.abspath(os.path.join(os.path.dirname('config.py'), '../Scripts'))
sys.path.append(config_path)
import config

# Import project-specific functions. 
# Python files (.py) have to be in same folder to work.
lib_path = os.path.abspath(os.path.join(os.path.dirname('Correspondence_XML_parser.py'), '../Scripts'))
sys.path.append(lib_path)

from Correspondence_XML_parser import *

# Ignore warnings related to deprecated functions.
warnings.filterwarnings('ignore')

url = 'https://dsg.xmldb-dev.northeastern.edu/BaseX964/rest/psc/'
user = config.username
pw = config.password

# Define tokenizer.
def fast_tokenize(text):
    
    # Get a list of punctuation marks
    punct = string.punctuation + '“' + '”' + '‘' + "’"
    
    lower_case = text.lower()
    lower_case = lower_case.replace('—', ' ').replace('\n', ' ')
    
    # Iterate through text removing punctuation characters
    no_punct = "".join([char for char in lower_case if char not in punct])
    
    # Split text over whitespace into list of words
    tokens = no_punct.split()
    
    return tokens

In [2]:
gensim.__version__

'4.0.1'

## Gather XML Files

In [3]:
%%time

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/"

input_directory = "Data/PSC/Taney/RBT_RawXML/*/*.xml"

# Gather all .xml files using glob.
files = glob.glob(abs_dir + input_directory)

CPU times: user 1.57 ms, sys: 1.92 ms, total: 3.49 ms
Wall time: 2.89 ms


In [4]:
# %%time

# # Must be connected to Northeastern's VPN.
# r = requests.get(url, 
#                  auth = (user, pw), 
#                  headers = {'Content-Type': 'application/xml'}
#                 )

# # Read in contents of pipeline.
# soup = BeautifulSoup(r.content, 'html.parser')

# # Split soup's content by \n (each line is a file path to an XML doc).
# # Use filter() to remove empty strings ('').
# # Convert back to list using list().
# files = list(filter(None, soup.text.split('\n')))

# # Filter list and retrieve only jqa/ files.
# files = [i for i in files if '......./' in i]

# len(files)

## Build Dataframe from XML

In [5]:
%%time
%%time

# Build dataframe from XML files.
# build_dataframe() called from Correspondence_XML_parser
df = build_dataframe(files)

df.head(3)

CPU times: user 73.5 ms, sys: 15.5 ms, total: 89 ms
Wall time: 124 ms
CPU times: user 73.8 ms, sys: 15.5 ms, total: 89.3 ms
Wall time: 124 ms


,file,date,source,target,subjects,references,text
0,RBT00099-verification.xml,1833-05-05,RBT,Ellicott-Thomas,,"jackson-andrew,kendall-amos,mickle-robert,tane...",Washington May 5. 1833My Dear Sir I received y...
1,RBT00146-verification.xml,1834-03-30,RBT,ellicott-thomas,,mccubbin-george,Washington March 30. 1834My Dear Sir I have on...
2,RBT01364-verification.xml,1833-08-05,RBT,jackson-andrew,,,Washington Augt. 5. 1833 My Dear Sir After ref...


## Build w2v Model

In [6]:
%%time

# Convert dataframe text field to list of sentences.
sentences = [sentence for text in df['text'] for sentence in sent_tokenize(text)]
words_by_sentence = [fast_tokenize(sentence) for sentence in sentences]
words_by_sentence = [sentence for sentence in words_by_sentence if sentence != []]

# Get total number of words and unique words.
single_list_of_words = []
for l in words_by_sentence:
    for w in l:
        single_list_of_words.append(w)
print (f'Word total: {len(single_list_of_words)}\nUnique word total {len(set(single_list_of_words))}')

# Build model.
model = gensim.models.Word2Vec(words_by_sentence, window=5, vector_size=100,
                               min_count=1, sg=1, alpha=0.025, batch_words=10000, workers=4)

# Unused arguments:
# size=100, iter=5,

# Save model for later use
model.wv.save_word2vec_format(abs_dir + '/Data/Output/WordVectors/taney_w2v.txt')

Word total: 60345
Unique word total 4490
CPU times: user 1.57 s, sys: 48.4 ms, total: 1.62 s
Wall time: 895 ms


## Analysis

In [7]:

print ('Words most similar to "independent":\n', [word for word, score in model.wv.most_similar(['independent'])], '\n')

print ('Words most similar to "food":\n', [word for word, score in model.wv.most_similar(['food'])], '\n')


Words most similar to "independent":
 ['party', 'number', 'admitted', 'cause', 'furnished', 'fall', 'gentleman', 'notwithstanding', 'reserve', 'patronage'] 



KeyError: "Key 'food' not present"